In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import tensorflow as tf

2025-12-02 13:19:16.572256: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-02 13:19:16.617943: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-02 13:19:16.709862: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-02 13:19:16.709904: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-02 13:19:16.714343: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

In [ ]:
from tensorflow_privacy.privacy.optimizers import dp_optimizer
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy

# build a simple CNN (4 convolutional layers) for CIFAR-10
# adding differential privacy using tensorflow-privacy
# uses already-loaded numpy arrays: x_train, y_train, x_test, y_test


# normalize copies of the data (don't overwrite originals)
x_train_norm = x_train.astype("float32") / 255.0
x_test_norm  = x_test.astype("float32")  / 255.0
y_train_flat = y_train.ravel()
y_test_flat  = y_test.ravel()


model = keras.models.Sequential([
    keras.layers.Input(shape=(32, 32, 3)),

    # Conv block 1 (2 conv layers)
    keras.layers.Conv2D(32, (3, 3), padding="same", activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(32, (3, 3), padding="same", activation="relu"),
    keras.layers.MaxPooling2D((2, 2)),

    # Conv block 2 (2 conv layers)
    keras.layers.Conv2D(64, (3, 3), padding="same", activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, (3, 3), padding="same", activation="relu"),
    keras.layers.MaxPooling2D((2, 2)),

    # classification head
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10)
])

In [ ]:
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer

# Create DP optimizer
optimizer = DPKerasSGDOptimizer(
    l2_norm_clip=1.5,
    noise_multiplier=1.3,
    num_microbatches=4,
    learning_rate=0.25
)

# Compile model normally
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=True,
        reduction=tf.losses.Reduction.NONE  # Important for DP!
    ),
    metrics=['accuracy']
)

# Train normally with fit()
history = model.fit(
    x_train_norm, 
    y_train_flat,
    batch_size=64,
    epochs=10,
    validation_data=(x_test_norm, y_test_flat),
    shuffle=True
)

Epoch 1/10


ValueError: in user code:

    File "/home/afruge/projects/privacy-preserving-ML/tf_privacy_venv/lib64/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/home/afruge/projects/privacy-preserving-ML/tf_privacy_venv/lib64/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/afruge/projects/privacy-preserving-ML/tf_privacy_venv/lib64/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/home/afruge/projects/privacy-preserving-ML/tf_privacy_venv/lib64/python3.11/site-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/afruge/projects/privacy-preserving-ML/tf_privacy_venv/lib64/python3.11/site-packages/keras/sr

In [ ]:
def plot_training_history(history=None, figsize=(12, 4), suptitle=None, save_path=None):
    """
    Plot training/validation accuracy and loss per epoch.
    If `history` is None, uses the notebook variable `history` if present.
    Returns (fig, axes).
    """
    if history is None:
        history = globals().get("history")
        if history is None:
            raise ValueError("No history provided and no global 'history' found.")
    # accept either Keras History object or a plain dict
    h = history.history if hasattr(history, "history") else history

    # determine number of epochs
    first_series = next(iter(h.values()))
    epochs = range(1, len(first_series) + 1)

    fig, axes = plt.subplots(1, 2, figsize=figsize)

    # Accuracy plot (handles both "accuracy" and legacy "acc")
    acc_key = "accuracy" if "accuracy" in h else ("acc" if "acc" in h else None)
    axes[0].plot(epochs, h[acc_key], "bo-", label="train")
    val_acc_key = "val_" + acc_key
    if val_acc_key in h:
        axes[0].plot(epochs, h[val_acc_key], "ro-", label="val")

    axes[0].axhline(y=0.7, color="gray", linestyle="--", label="required baseline accuracy")
    axes[0].set_title("Accuracy")
    axes[0].set_xlabel("Epoch")
    axes[0].set_ylabel(acc_key)
    axes[0].legend()

    # Loss plot
    axes[1].plot(epochs, h["loss"], "bo-", label="train")
    if "val_loss" in h:
        axes[1].plot(epochs, h["val_loss"], "ro-", label="val")
    axes[1].set_title("Loss")
    axes[1].set_xlabel("Epoch")
    axes[1].set_ylabel("loss")
    axes[1].legend()

    if suptitle:
        fig.suptitle(suptitle)
    plt.tight_layout()

    if save_path:
        fig.savefig(save_path, bbox_inches="tight")

    plt.show()
    return fig, axes

plot_training_history(history, suptitle="Training History for CIFAR-10 CNN")

In [ ]:
model.save("dp_model.keras")